In [7]:
import pandas as pd
import numpy as np
import imp

In [8]:
import warnings
warnings.filterwarnings("ignore")

In [9]:
raw_dat = pd.read_excel("../data/raw_data.xlsx")
raw_dat['Normalized'] = (raw_dat['mkt_r'] - raw_dat['dep_f_r'])/(raw_dat['len_f_r'] - raw_dat['dep_f_r'])
dat = raw_dat.drop(columns=['mkt_r', 'dep_f_r', 'len_f_r'])
# dat.to_csv("../data/reserve_demand_cruve_data.csv", index=False)

In [10]:
dat = pd.read_csv("../data/reserve_demand_cruve_data.csv")

In [19]:
import reserves_demand.curve_fit as cf
imp.reload(cf)

obj = cf.CurveFitter(
    data = dat, 
    dep_var = 'Normalized',
    main_indep_var= 'excess_reserve',
    dep_var_name= 'Normalized Overnight Rate',                 
    main_indep_var_name= 'Excess Reserve',
    dummy_col =None,
    constant = True,
    Q = 0.9,
    nFolds=5,
    parallel=True,
    search_method = 'all_combinations'
)
best_curve, cv_result = obj.variable_select(on='RMSE')

Running cross-validation for logistic curve


100%|██████████| 63/63 [00:40<00:00,  1.57 it/s]

Best combination for logistic: ['excess_reserve', 'vix', 'acnusd']
Training Metrics:
RMSE         0.060783
MAE          0.038470
MAPE        22.611494
R2           0.911974
MSLE         0.002417
MedianAE     0.018761
dtype: float64

Validation Metrics:
RMSE         0.060734
MAE          0.038908
MAPE        22.787613
R2           0.909969
MSLE         0.002459
MedianAE     0.019129
dtype: float64


Running cross-validation for redLogistic curve



 87%|████████▋ | 55/63 [00:26<00:03,  2.31 it/s]

In [ ]:
best_params = obj.fit_best_curves()

{'logistic': {'vars': ['excess_reserve', 'vix', 'acnusd'],
  'avg': array([ 3.20862816e-02,  9.54462640e-01,  2.37994910e-02, -2.27238351e+00,
         -3.52856206e-03, -1.83319230e-03]),
  'upper': array([-0.11604193,  0.74296785,  0.18215942, -2.37268304, -0.01149307,
         -0.03882772]),
  'lower': array([-0.66527827,  0.51100519,  0.57578265, -2.16060889,  0.0345727 ,
         -0.89591137])},
 'redLogistic': {'vars': ['excess_reserve', 'vix', 'hicp'],
  'avg': array([-0.94472063,  0.34344783, -9.21034037, -0.15019342, -0.1228711 ]),
  'upper': array([-1.01137884,  0.38857342, -1.60017887, -0.02319583, -0.20243817]),
  'lower': array([-1.11950121,  0.65019221, -1.00340151, -1.60154613, -1.36027736])},
 'fixLogistic': {'vars': ['constant', 'excess_reserve', 'vix', 'hicp'],
  'avg': array([-0.94471235, -1.06876642, -9.21034037, -0.1500971 , -0.12283774]),
  'upper': array([-0.93871238, -1.05038029, -4.33849005, -0.06718281, -0.13718213]),
  'lower': array([-1.1547451 , -0.47720246,

In [ ]:
perf_metrics_df, param_df = obj.compare_curves()

KeyError: 'up'